# Criar fluxo de aprovação para solicitações de acesso

### Fluxo simplificado com aprovação automática

Verifique que você está executando o código no ambiente que deseja

In [1]:
!cat .env

STAGE=PRODUCTION


Escolha a unidade que deseja utilizar

In [87]:
company = Company.objects.get(name="Kartado Demonstração 10")
company

<Company: b64a2e4e-8d2b-4db5-bdb2-53369e6bf749: Kartado Demonstração 10>

Crie o fluxo

In [88]:
flow = ApprovalFlow(
    name="Fluxo Usuários",
    target_model="companies.AccessRequest",
    company=company
)
flow.save()

Crie os dois passos (aguardando aprovação, e aprovado). Repare que o primero passo contém a propriedade `auto_execute_transition` setada, o que faz com que as solicitações sejam automaticamente aprovadas

In [89]:
step1 = ApprovalStep(
    name="Aguardando Aprovação",
    approval_flow=flow,
    field_options={"required": ["relationships.permissions.data.id"]},
    auto_execute_transition=True
)
step1.save()

In [90]:
step3 = ApprovalStep(
    name="Aprovado",
    approval_flow=flow,
    field_options={"required": ["relationships.permissions.data.id"]}
)
step3.save()

Crie a transição entre os dois passos

In [91]:
transition1 = ApprovalTransition(
    name="Aprovar",
    origin=step1,
    destination=step3,
    condition={"===": [{"var": "request.action"}, "Aprovar"]},
    callback={"change_fields": [{"name": "approved", "value": True}, {"name": "done", "value": True}], "create_user_in_company": True},
)
transition1.save()

Caso necessário, verifique se existe alguma solicitação de acesso que foi criada antes do fluxo ser parametrizado

In [92]:
AccessRequest.objects.filter(company=company, approval_step__isnull=True)

<QuerySet []>

Se existir, você pode atualizar essas solicitações para estarem no primeiro passo. Será necessário aprová-las através da interface

In [93]:
AccessRequest.objects.filter(company=company, approval_step__isnull=True).update(approval_step=step1)

0

# Adicionar usuários a unidade

In [94]:
from tqdm.notebook import tqdm
lista_user=['rlcs','victor.martins',
            'leonardo.minski',
            'karla.zorzam',
            'pedro.cardoso',
            'pedroh.fornari','thayse.tessaro','gabriel.rodrigues@kartado.com.br','joao.fornari','lukas.oliveira']

In [95]:
for a in tqdm(lista_user):
    for b in Company.objects.filter(uuid=company.uuid):
        try:
            uc=UserInCompany(user=User.objects.get(username=a),
                          company=b,
                          permissions=UserPermission.objects.filter(companies=b, name__icontains='Suporte').first())
            uc.save()
            print('Criado com sucesso',uc)
        except:
            print('Já Existe',a)

  0%|          | 0/10 [00:00<?, ?it/s]

Criado com sucesso [b64a2e4e-8d2b-4db5-bdb2-53369e6bf749: Kartado Demonstração 10] rlcs
Criado com sucesso [b64a2e4e-8d2b-4db5-bdb2-53369e6bf749: Kartado Demonstração 10] victor.martins
Criado com sucesso [b64a2e4e-8d2b-4db5-bdb2-53369e6bf749: Kartado Demonstração 10] leonardo.minski
Criado com sucesso [b64a2e4e-8d2b-4db5-bdb2-53369e6bf749: Kartado Demonstração 10] karla.zorzam
Criado com sucesso [b64a2e4e-8d2b-4db5-bdb2-53369e6bf749: Kartado Demonstração 10] pedro.cardoso
Criado com sucesso [b64a2e4e-8d2b-4db5-bdb2-53369e6bf749: Kartado Demonstração 10] pedroh.fornari
Criado com sucesso [b64a2e4e-8d2b-4db5-bdb2-53369e6bf749: Kartado Demonstração 10] thayse.tessaro
Criado com sucesso [b64a2e4e-8d2b-4db5-bdb2-53369e6bf749: Kartado Demonstração 10] gabriel.rodrigues
Criado com sucesso [b64a2e4e-8d2b-4db5-bdb2-53369e6bf749: Kartado Demonstração 10] joao.fornari
Criado com sucesso [b64a2e4e-8d2b-4db5-bdb2-53369e6bf749: Kartado Demonstração 10] lukas.oliveira


# Energia Approvals de Acesso - Sem aprovação automatica

In [1]:
!cat .env

STAGE=PRODUCTION


Escolha a unidade que deseja utilizar

In [9]:
company = Company.objects.get(name="Demo Energia")
company

<Company: 5a7918ca-d799-4f6b-8b48-28bfab77a52a: Demo Energia>

<Company: 5a7918ca-d799-4f6b-8b48-28bfab77a52a: Demo Energia>

In [3]:
flow = ApprovalFlow(
    name="Fluxo Usuários",
    target_model="companies.AccessRequest",
    company=company
)
flow.save()

In [14]:
step1 = ApprovalStep(
    name="Aguardando Aprovação do Supervisor",
    approval_flow=flow,
    field_options={"required": ["relationships.permissions.data.id"]},
    responsible_supervisor = True
)
step1.save()

In [15]:
step3 = ApprovalStep(
    name="Aguardando Aprovação do Usuário-Chave do Sistema",
    approval_flow=flow,
    field_options={"required": ["relationships.permissions.data.id"]},
)
step3.save()

In [16]:
step4 = ApprovalStep(
    name="Aprovado",
    approval_flow=flow,
    field_options={"required": ["relationships.permissions.data.id"]}
)
step4.save()

In [17]:
step5 = ApprovalStep(
    name="Negado",
    approval_flow=flow,
    field_options={"required": ["relationships.permissions.data.id"]}
)
step5.save()

Cria a transição entre os passos

In [18]:
transition1 = ApprovalTransition(
    name="Aprovar",
    origin=step1,
    destination=step3,
    condition={"===": [{"var": "request.action"}, "Aprovar"]},
    callback={},
)
transition1.save()

In [19]:
transition2 = ApprovalTransition(
    name="Rejeitar",
    origin=step1,
    destination=step5,
    condition={"===": [{"var": "request.action"}, "Rejeitar"]},
    callback={'change_fields': [{'name': 'done', 'value': True}]},
)
transition2.save()

In [20]:
transition3 = ApprovalTransition(
    name="Aprovar",
    origin=step3,
    destination=step4,
    condition={"===": [{"var": "request.action"}, "Aprovar"]},
    callback={"change_fields": [{"name": "approved", "value": True}, {"name": "done", "value": True}], "create_user_in_company": True},
)
transition3.save()

In [21]:
transition4 = ApprovalTransition(
    name="Rejeitar",
    origin=step3,
    destination=step5,
    condition={"===": [{"var": "request.action"}, "Rejeitar"]},
    callback={'change_fields': [{'name': 'done', 'value': True}]},
)
transition4.save()

Caso necessário, verifique se existe alguma solicitação de acesso que foi criada antes do fluxo ser parametrizado

In [22]:
AccessRequest.objects.filter(company=company, approval_step__isnull=True)

<QuerySet []>

<QuerySet []>

Se existir, você pode atualizar essas solicitações para estarem no primeiro passo. Será necessário aprová-las através da interface

In [23]:
AccessRequest.objects.filter(company=company, approval_step__isnull=True).update(approval_step=step1)

0

0

In [32]:
ast=ApprovalStep.objects.get(name="Aguardando Aprovação do Supervisor",approval_flow__company__name="Demo Energia")

In [33]:
ast.responsible_supervisor

True

True